# Packages e import

In [3]:
!pip install tensorflow
!pip install matplotlib
!pip install kaggle

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
import os

# Scaricamento dataset e estrazione

In [5]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
!kaggle datasets download -d kritikseth/fruit-and-vegetable-image-recognition
!unzip /content/fruit-and-vegetable-image-recognition.zip -d /content/dataset

Dataset URL: https://www.kaggle.com/datasets/kritikseth/fruit-and-vegetable-image-recognition
License(s): CC0-1.0
 99% 1.97G/1.98G [00:29<00:00, 122MB/s]
100% 1.98G/1.98G [00:29<00:00, 73.1MB/s]
Archive:  /content/fruit-and-vegetable-image-recognition.zip
  inflating: /content/dataset/test/apple/Image_1.jpg  
  inflating: /content/dataset/test/apple/Image_10.jpg  
  inflating: /content/dataset/test/apple/Image_2.jpg  
  inflating: /content/dataset/test/apple/Image_3.jpg  
  inflating: /content/dataset/test/apple/Image_4.jpg  
  inflating: /content/dataset/test/apple/Image_5.jpg  
  inflating: /content/dataset/test/apple/Image_6.JPG  
  inflating: /content/dataset/test/apple/Image_7.jpg  
  inflating: /content/dataset/test/apple/Image_8.jpg  
  inflating: /content/dataset/test/apple/Image_9.jpg  
  inflating: /content/dataset/test/banana/Image_1.jpg  
  inflating: /content/dataset/test/banana/Image_10.jpg  
  inflating: /content/dataset/test/banana/Image_2.jpg  
  inflating: /content/da

# Preprocessing immagini

In [ ]:
train_dir = '/content/dataset/train'
val_dir = '/content/dataset/validation'
test_dir = '/content/dataset/test'

img_height, img_width = 224, 224

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical')

val_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Labels

In [ ]:
print(test_generator.classes)
class_labels = list(test_generator.class_indices.keys())
print(class_labels)
print(len(class_labels))

# Controllo se GPU è disponibile

In [ ]:
print("GPU disponibile:" , "sì" if tf.config.list_physical_devices('GPU') else "no")

# Importazione del modello senza output layer

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height,img_width,3))
base_model.trainable = False # Blocco la possibilità di allenare tutti i layers

# Creazione custom layers e compilazione modello

In [17]:
inputs = base_model.input

x = GlobalAveragePooling2D()(base_model.output)

x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(36, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


**Creazione e allenamento del modello**

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=20
)

**Conversione in TensorflowLite**

In [ ]:
model.save('Fruit_classifier.h5')

# Funzione di conversione
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Applica ottimizzazioni (se desiderato)
tflite_model = converter.convert()

# Salvo il modello convertito in un file
with open('Fruit_classifier.tflite', 'wb') as f:
    f.write(tflite_model)